# A tool to evaluate a mathematical expression

This week the tool used in FlightAI was a database lookup function.

Here I implement a python code interpreter function as tool.

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()
MODEL = "gpt-4o-mini"

# Defining the tool function

Add print statements to make sure the function is used instead of the native gpt interpreter capability.

I used multi shot in the system prompt to make sure gpt generate the code in the format that the tool accept.

In [ ]:
def evaluate_math_expression(my_code):
    print(f"EXECUTING FUNCTION WITH CODE: {my_code}")
    exec(my_code)
    r = locals()['interpreter_result'] 
    return r


math_function = {
    "name": "evaluate_math_expression",
    "description": "Give the result of a math expression. \
    Call this whenever you need to know the result of a mathematical expression. \
    Generate python code ALWAYS with the final result assigned to a variable called 'interpreter_result'. \
    For example when a user asks 'What is 2+2' generate 'interpreter_result = 2+2', and pass this code to the tool. \
    Another example if a user ask 'What is log(5)' generate 'import math; interpreter_result = math.log(5)' and pass this code to the tool.",
    
    "parameters": {
        "type": "object",
        "properties": {
            "my_code": {
                "type": "string",
                "description": "The python math expression to evaluate",
            },
        },
        "required": ["my_code"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": math_function}]

In [ ]:
evaluate_math_expression("import math; interpreter_result = math.log(5)")

# Using the tool in a UI program

You can ask messages like:
- "What is 2+2?"
- "What is 3 power 2?"
- "I have 25 apples. I buy 10 apples. How manny apples do I have?"

In [ ]:
system_message = "You are a math assistant. \
Generate python code to give result of a math expression, always name the result 'interpreter_result'. \
For example when a user asks 'What is 2+2', generate 'interpreter_result = 2+2' and pass this code to the tool. \
Another example: if a user ask 'What is log(5)' generate 'import math; interpreter_result = math.log(5)'"

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        response = handle_tool_call(message)
        print(response)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    my_code = arguments.get('my_code')
    interpreter_result = evaluate_math_expression(my_code)
    response = {
        "role": "tool",
        "content": json.dumps({"my_code": my_code,"interpreter_result": interpreter_result}),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()